## Implementing Oversampling technique (SMOTE)

**This notebook is to demonstrate oversampling technique applied to handle imbalanced dataset.**

The resulting dataframe that came out of the SMOTE implementation was then applied to 2 models - Logistic regression and Random Forest. However, due to the limited cluster memory availability, both model run could not succeed and aborted with this error message - "**Error: java.lang.OutOfMemoryError: GC overhead limit exceeded**"

We are attaching this notebook just as a reference to showcase the implementation logic for SMOTE technique.

### Section 1 - Setup Environment

In [0]:
import random
from functools import reduce
from pyspark.sql import Row
from pyspark.sql.functions import rand,udf,lower,sum as ps_sum,count as ps_count,row_number
from pyspark.sql.window import *
from pyspark.sql import DataFrame
from pyspark.ml.feature import VectorAssembler,BucketedRandomProjectionLSH,VectorSlicer
from pyspark.sql.window import Window
from pyspark.ml.linalg import Vectors,VectorUDT
from pyspark.sql.functions import array, create_map, struct

from pyspark.sql.functions import col, concat, count, countDistinct, expr, lit, length, max as pyspark_max, min as pyspark_min, mean, substring, split, sum as pyspark_sum, when, to_utc_timestamp, to_timestamp, unix_timestamp, isnan
import pandas as pd
from html import escape
from IPython.display import HTML, display as ipython_display
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from pyspark.sql import SQLContext
from pyspark.sql.types import IntegerType, StringType, BooleanType, DateType, DoubleType
from pandas.tseries.holiday import USFederalHolidayCalendar
from pyspark.sql.functions import substring
from pyspark.sql import functions as F

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import col, max, substring
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.mllib.util import MLUtils
from pyspark.ml.feature import Imputer
from pyspark.ml.classification import LinearSVC as svc
from pyspark.ml import Pipeline

import numpy as np
import itertools
import time

In [0]:
blob_container = "w261-container" # The name of your container created in https://portal.azure.com
storage_account = "w261storageaccount" # The name of your Storage account created in https://portal.azure.com
secret_scope = "w261scope" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "w261key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

spark.conf.set(
  f"fs.azure.sas.{blob_container}.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

### Section 2 - Load Data

In [0]:
# Reading the split test and train data from blob storage in case of full data
# Reading the toy test and train data from blob storage in case of toy dataset

train_df = spark.read.parquet(f"{blob_url}/original_train_data_full_v2/*")
test_df = spark.read.parquet(f"{blob_url}/original_test_data_full_v2/*")

In [0]:
#Preparing final dataset for modeling
# Let's find out the datatype of the features, which will be the same for both train and test data

print("\n----Categorical columns train-----")
cat_cols = [x for (x, dataType) in train_df.dtypes if dataType == "string"]
print(cat_cols)

print("\n----Numeric columns train-----")
num_cols = [x for (x, dataType) in train_df.dtypes if (((dataType == "double") or (dataType == "int")) & (x != "departure_delay_boolean"))]
print(num_cols)

----Categorical columns train-----
['year', 'month', 'day_of_week', 'hour', 'carrier', 'holiday', 'weather_obs_prev', 'weather_obs_prev_2', 'origin', 'destination', 'prev_dep_delayed_confirmed', 'wnd_type_prev', 'vis_var_prev']

----Numeric columns train-----
['planned_time_between_flights', 'distance', 'wnd_angle_prev', 'wnd_speed_prev_2', 'wnd_speed_prev', 'wnd_cloud_angle_prev', 'vis_dist_prev', 'vis_dist_prev_2', 'tmp_c_prev', 'dew_c_prev', 'route_delay', 'origin_arrival_delay', 'origin_departure_delay', 'dest_departure_delay', 'dest_arrival_delay']

In [0]:
# One hot encoding - converting categorical columns into binary sparse vectors

from pyspark.ml.feature import (OneHotEncoder, StringIndexer)
string_indexer = [
    StringIndexer(inputCol=x, outputCol=x + "_StringIndexer", handleInvalid="skip")
    for x in cat_cols
]

one_hot_encoder = [
    OneHotEncoder(
        inputCols=[f"{x}_StringIndexer" for x in cat_cols],
        outputCols=[f"{x}_OneHotEncoder" for x in cat_cols],
    )
]

from pyspark.ml.feature import Imputer

imputer = [
      Imputer(inputCol=x, outputCol= x + "_Imputed").setStrategy("median")
      for x in num_cols
]


from pyspark.ml.feature import StandardScaler  

scaler_vector_assembler = VectorAssembler(inputCols = [f"{x}_Imputed" for x in num_cols], outputCol="numeric_vec")
scaler_vector_assembler.setHandleInvalid('keep')

scaler = StandardScaler(inputCol="numeric_vec", outputCol="scaled_features")

In [0]:
from pyspark.ml.feature import VectorAssembler

assembler_input = ["scaled_features"]
assembler_input += [f"{x}_OneHotEncoder" for x in cat_cols]

vector_assembler = VectorAssembler(
    inputCols=assembler_input, outputCol="VectorAssembler_features"
)

### Section 3 - Imbalanced data handling

#### 3.0.  Setting up Pipeline

In [0]:
stages = []
stages += imputer
stages += [scaler_vector_assembler , scaler]
stages += string_indexer
stages += one_hot_encoder
stages += [vector_assembler]

In [0]:
from pyspark.ml import Pipeline

general_pipeline = Pipeline().setStages(stages)
pipeline_model = general_pipeline.fit(train_df)
model_data_test = pipeline_model.transform(test_df)
model_data_train = pipeline_model.transform(train_df) 

selectedcols = ["departure_delay_boolean", "VectorAssembler_features"]
model_data_train = model_data_train.select(selectedcols)
model_data_test = model_data_test.select(selectedcols)

In [0]:
model_data_train.write.mode('overwrite').parquet(f"{blob_url}/train_data_pre_upsample")
model_data_test.write.mode('overwrite').parquet(f"{blob_url}/test_data_pre_upsample")
train_df = spark.read.parquet(f"{blob_url}/train_data_pre_upsample/*")
test_df = spark.read.parquet(f"{blob_url}/test_data_pre_upsample/*")

#### 3.1. Oversampling (SMOTE)
SMOTE is a method to handle imbalanced data and is applied to upsample the minority class dataset by creating synthetic samples.

In [0]:
# SMOTE Implementation for handling imbalanced data between majority and minority class

def smote(df):
    '''
    contains logic to perform smote oversampling, given a spark df with 2 classes
    inputs:
    * df: cat cols are already stringindexed, num cols are assembled into 'VectorAssembler_features' vector
      df target col should be 'departure_delay_boolean'
    output:
    * oversampled_df: spark df after smote oversampling
    '''
    dataInput_min = df[df['departure_delay_boolean'] == 1]
    dataInput_maj = df[df['departure_delay_boolean'] == 0]
    
    # seed = Random, 12345 or 40
    # bucketLength = math.pow(numRecords, -1/inputDim) = math.pow(5017444, -1/26)
     # LSH, bucketed random projection
    brp = BucketedRandomProjectionLSH(inputCol="VectorAssembler_features", outputCol="hashes",seed=12345, bucketLength=0.55)
    # smote only applies on existing minority instances    
    model = brp.fit(dataInput_min)
    model.transform(dataInput_min)

    # here distance is calculated from brp's param inputCol
    self_join_w_distance = model.approxSimilarityJoin(dataInput_min, dataInput_min, float("inf"), distCol="EuclideanDistance")

    # remove self-comparison (distance 0)
    self_join_w_distance = self_join_w_distance.filter(self_join_w_distance.EuclideanDistance > 0)

    over_original_rows = Window.partitionBy("datasetA").orderBy("EuclideanDistance")

    self_similarity_df = self_join_w_distance.withColumn("r_num", F.row_number().over(over_original_rows))

    self_similarity_df_selected = self_similarity_df.filter(self_similarity_df.r_num <= 2)

    over_original_rows_no_order = Window.partitionBy('datasetA')

    # list to store batches of synthetic data
    res = []
    
    # two udf for vector add and subtract, subtraction include a random factor [0,1]
    subtract_vector_udf = F.udf(lambda arr: random.uniform(0, 1)*(arr[0]-arr[1]), VectorUDT())
    add_vector_udf = F.udf(lambda arr: arr[0]+arr[1], VectorUDT())
    
    # retain original columns
    original_cols = dataInput_min.columns
    # range_multiplier = dataInput_maj / dataInput_min
    range_multiplier = 10
    for i in range(range_multiplier):
        print("generating batch %s of synthetic instances"%i)
        # logic to randomly select neighbour: pick the largest random number generated row as the neighbour
        df_random_sel = self_similarity_df_selected.withColumn("rand", F.rand()).withColumn('max_rand', F.max('rand').over(over_original_rows_no_order))\
                            .where(F.col('rand') == F.col('max_rand')).drop(*['max_rand','rand','r_num'])
        # create synthetic feature numerical part
        df_vec_diff = df_random_sel.select('*', subtract_vector_udf(F.array('datasetA.VectorAssembler_features', 'datasetB.VectorAssembler_features')).alias('vec_diff'))
        df_vec_modified = df_vec_diff.select('*', add_vector_udf(F.array('datasetA.VectorAssembler_features', 'vec_diff')).alias('VectorAssembler_features'))
        
        # for categorical cols, either pick original or the neighbour's cat values
        for c in original_cols:
            # randomly select neighbour or original data
            col_sub = random.choice(['datasetA','datasetB'])
            val = "{0}.{1}".format(col_sub,c)
            if c != 'VectorAssembler_features':
                # do not unpack original numerical features
                df_vec_modified = df_vec_modified.withColumn(c,F.col(val))
        
        # this df_vec_modified is the synthetic minority instances,
        df_vec_modified = df_vec_modified.drop(*['datasetA','datasetB','vec_diff','EuclideanDistance'])
        
        res.append(df_vec_modified)
    
    dfunion = reduce(DataFrame.unionAll, res)
    # union synthetic instances with original full (both minority and majority) df
    oversampled_df = dfunion.union(df.select(dfunion.columns))
    
    return oversampled_df

In [0]:
oversampled_train_df = smote(model_data_train)

generating batch 0 of synthetic instances
generating batch 1 of synthetic instances
generating batch 2 of synthetic instances
generating batch 3 of synthetic instances
generating batch 4 of synthetic instances
generating batch 5 of synthetic instances
generating batch 6 of synthetic instances
generating batch 7 of synthetic instances
generating batch 8 of synthetic instances
generating batch 9 of synthetic instances

In [0]:
oversampled_train_df.write.mode('overwrite').parquet(f"{blob_url}/train_data_upsample")
train_df = spark.read.parquet(f"{blob_url}/train_data_upsample/*")

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2297543790346889> in <module> 
 ----> 1 oversampled_train_df . write . mode ( 'overwrite' ) . parquet ( f"{blob_url}/train_data_upsample" ) 
 2 train_df = spark . read . parquet ( f"{blob_url}/train_data_upsample/*" ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, path, mode, partitionBy, compression) 
 1275 self . partitionBy ( partitionBy ) 
 1276 self . _set_opts ( compression = compression ) 
 -> 1277 self . _jwrite . parquet ( path ) 
 1278 
 1279 def text ( self , path , compression = None , lineSep = None ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o6374.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:307)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:194)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:121)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:119)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:144)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:213)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:257)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:165)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:253)
	at org.apache.spark.sql.execution.SparkPlan.execute(SparkPlan.scala:209)
	at org.apache.spark.sql.execution.QueryExecution.toRdd$lzycompute(QueryExecution.scala:167)
	at org.apache.spark.sql.execution.QueryExecution.toRdd(QueryExecution.scala:166)
	at org.apache.spark.sql.DataFrameWriter.$anonfun$runCommand$1(DataFrameWriter.scala:1079)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$5(SQLExecution.scala:130)
	at org.apache.spark.sql.execution.SQLExecution$.withSQLConfPropagated(SQLExecution.scala:273)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withCustomExecutionEnv$1(SQLExecution.scala:104)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:854)
	at org.apache.spark.sql.execution.SQLExecution$.withCustomExecutionEnv(SQLExecution.scala:77)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:223)
	at org.apache.spark.sql.DataFrameWriter.runCommand(DataFrameWriter.scala:1079)
	at org.apache.spark.sql.DataFrameWriter.saveToV1Source(DataFrameWriter.scala:468)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:438)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:303)
	at org.apache.spark.sql.DataFrameWriter.parquet(DataFrameWriter.scala:964)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.

### Section 4 - ML section

#### 4.0. Logistic regression

In [0]:
from pyspark.ml.classification import LogisticRegression
# Setup pipeline

start_lr = time.time()

lr = LogisticRegression(featuresCol='VectorAssembler_features',
                        labelCol='departure_delay_boolean', 
                        regParam = 0.2, 
                        fitIntercept=True, 
                        elasticNetParam=0.0, 
                        maxIter=10, 
                        threshold = 0.6)

stages_lr = [lr]
pipeline_lr = Pipeline(stages=stages_lr)
lr_model = pipeline_lr.fit(train_df)
lrPredictions = lr_model.transform(test_df)

lrPredictions.write.mode('overwrite').parquet(f"{blob_url}/lrPredictions_upsample")

print(f'lr model completed job in {time.time() - start_lr} seconds.')
lr_time_model = time.time() - start_lr

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-671119174920103> in <module> 
 14 stages_lr = [ lr ] 
 15 pipeline_lr = Pipeline ( stages = stages_lr ) 
 ---> 16 lr_model = pipeline_lr . fit ( train_df ) 
 17 lrPredictions = lr_model . transform ( test_df ) 
 18 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 112 dataset = stage . transform ( dataset ) 
 113 else : # must be an Estimator 
 --> 114 model = stage . fit ( dataset ) 
 115 transformers . append ( model ) 
 116 if i < indexOfLastEstimator : 

 /databricks/python_shell/dbruntime/MLWorkloadsInstrumentation/_pyspark.py in patched_method (self, *args, **kwargs) 
 28 call_succeeded = False 
 29 try : 
 ---> 30 result = original_method ( self , * args , ** kwargs ) 
 31 call_succeeded = True 
 32 return result

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 333 
 334 def _fit ( self , dataset ) : 
 --> 335 java_model = self . _fit_java ( dataset ) 
 336 model = self . _create_model ( java_model ) 
 337 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 330 """
 331 self . _transfer_params_to_java ( ) 
 --> 332 return self . _java_obj . fit ( dataset . _jdf ) 
 333 
 334 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o8348.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 4 in stage 137806.0 failed 4 times, most recent failure: Lost task 4.3 in stage 137806.0 (TID 690574) (10.0.128.119 executor 155): scala.MatchError: [null,1.0,(804,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,23,33,40,73,77,78,79,98,494,797,799,802],[0.01227876010576003,0.4919111581895064,1.8347581029801274,2.5014659186006987,2.709770520485189,0.16987135815752624,4.853988524506112,4.830009665721882,2.028271771493124,1.7381782712661273,3.938640625574508,6.17684452651295,6.3059605298552395,5.114406777003982,4.582645043415024,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.PredictorParams.$anonfun$extractInstance

In [0]:
# Logistic Regression Model Evaluation with scikit learn

lrPredictions = spark.read.parquet(f"{blob_url}/lrPredictions_upsample/*")

y_true_lr = lrPredictions.select(['departure_delay_boolean']).collect()
y_pred_lr = lrPredictions.select(['prediction']).collect()

# Print metrics
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_true_lr, y_pred_lr))
print(confusion_matrix(y_true_lr, y_pred_lr))

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-671119174920105> in <module> 
 1 # Logistic Regression Model Evaluation with scikit learn 
 2 
 ----> 3 lrPredictions = spark . read . parquet ( f"{blob_url}/lrPredictions_upsample/*" ) 
 4 
 5 y_true_lr = lrPredictions . select ( [ 'departure_delay_boolean' ] ) . collect ( ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in parquet (self, *paths, **options) 
 483 int96RebaseMode=int96RebaseMode)
 484 
 --> 485 return self . _df ( self . _jreader . parquet ( _to_seq ( self . _spark . _sc , paths ) ) ) 
 486 
 487 def text(self, paths, wholetext=False, lineSep=None, pathGlobFilter=None,

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Path does not exist: wasbs://w261-container@w261storageaccount.blob.core.windows.net/lrPredictions_upsample/*